In [ ]:
import openai
import os

openai.api_key = os.environ["OPENAI_API_KEY"]

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [ ]:

event = {
  "apiVersion": "v1",
  "count": 666,
  "eventTime": "",
  "firstTimestamp": "2023-05-04T18:30:41Z",
  "involvedObject": {
      "apiVersion": "v1",
      "fieldPath": "spec.containers{container-0}",
      "kind": "Pod",
      "name": "crash-loop-7cd9546f4f-strcl",
      "namespace": "default",
      "resourceVersion": "52054",
      "uid": "952c7516-e645-4fc3-a64b-7edc69cdf902"
  },
  "kind": "Event",
  "lastTimestamp": "2023-05-04T18:50:27Z",
  "message": "Back-off restarting failed container",
  "metadata": {
      "creationTimestamp": "2023-05-04T18:30:41Z",
      "name": "crash-loop-7cd9546f4f-strcl.175c0441bc6c05ab",
      "namespace": "default",
      "resourceVersion": "52506",
      "uid": "4d3f51b8-639b-4821-8cb4-9545db08c222"
  },
  "reason": "BackOff",
  "reportingComponent": "",
  "reportingInstance": "",
  "source": {
      "component": "kubelet",
      "host": "lens-desktop-kube"
  },
  "type": "Warning"
 }

deployment = {
  "apiVersion": "apps/v1",
  "kind": "Deployment",
  "metadata": {
    "name": "crash-loop"
  },
  "spec": {
    "replicas": 1,
    "selector": {
      "matchLabels": {
        "app": "crash-loop"
      }
    },
    "template": {
      "metadata": {
        "labels": {
          "app": "crash-loop"
        }
      },
      "spec": {
        "containers": [
          {
            "name": "container-0",
            "image": "node:alpine",
            "command": ["node", "index.js"]
          }
        ]
      }
    }
  }
}

log = """
node:internal/modules/cjs/loader:1085
  throw err;
  ^

Error: Cannot find module '/index.js'
    at Module._resolveFilename (node:internal/modules/cjs/loader:1082:15)
    at Module._load (node:internal/modules/cjs/loader:928:27)
    at Function.executeUserEntryPoint [as runMain] (node:internal/modules/run_main:83:12)
    at node:internal/main/run_main_module:23:47 {
  code: 'MODULE_NOT_FOUND',
  requireStack: []
}

Node.js v20.0.0
"""
prompt = f"""
As a technical expert in Kubernetes, your task is to diagnose and resolve questions and issues related to
these technologies. You should have a deep understanding of the underlying principles
of Kubernetes objects, as well as experience troubleshooting
common problems that may arise.

You will be provided with a Kubernetes event (in json string ```{event}```) with the following values
- Event message ```{event["message"]}```
- Event reason ```{event["reason"]}```
- Event type ```{event["type"]}```
- Event involvedObject kind ```{event["involvedObject"]["kind"]}```
- Event invlivedObject name ```{event["involvedObject"]["name"]}```

And the logs from the invlivedObject ```{log}``` and the full json definition of the invlivedObject ```{event["involvedObject"]["name"]}```
in json ```{deployment}```

Your task is to perform the following actions, and provide them in JSON format with the following keys: 
rootCause, fixBykubectlCmd, fixBykubectlCmdDescription, kubeObjectName, diagnoseByKubectlCmd.
- Use at most 50 words to summarize the root cause of the event including the name and namespace of the relevent
kube objects. You should be able to communicate effectively with non-technical users, providing
clear explanations of complex concepts and solutions.
- If the problem is related to Kubernetes infrastructure, such as kubernetes network, persistent volume, kublet, 
docker or kubeproxy, ouput a kubectl command to fix the issues, if you determin the root cause is from the 
application code in the container and it is not possible to fix by kubectl, return an empty string.
- Use at most 50 words to describe the action perfrom in fixBykubectlCmd. You should be able to communicate 
effectively with non-technical users, providing clear explanations of complex concepts and solutions.
- The name of the kube object to perform the fixBykubectlCmd action to fix such issues.
- A kubectl command to diagnose other possible cause of the issues.
"""

response = get_completion(prompt)
print(response)